In [6]:
import numpy as np 
import cv2
import os
import imutils

In [2]:
labels={}
for i,Class in enumerate(os.listdir('I:/New/')):
    if Class in labels:
        pass
    elif Class!='.ipynb_checkpoints' and Class!='model':
        labels[Class]=i
labels      

{'non tumor': 0, 'tumor': 1}

In [3]:
def crop_imgs(img, add_pixels_value=0):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

     # find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    ADD_PIXELS = add_pixels_value
    new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    s1,s2,_=new_img.shape
    mask = np.zeros_like(new_img)
# create a white filled ellipse
    mask=cv2.ellipse(mask, center=(s2//2, s1//2), axes=((s2-20)//2,(s1-5)//2),
         angle=0, startAngle=0, endAngle=360, color=(255,255), thickness=-1)
    result = np.bitwise_and(new_img,mask)  
#       result=cv2.cvtColor(result, cv2.COLOR_BGR2RGB)


    return np.array(result)


In [4]:
def resize_img(img, img_size):
    img = cv2.resize(
        img,
        dsize=img_size,
        interpolation=cv2.INTER_CUBIC
    )
    return img

In [10]:
Img_data=[]
Img_label=[]
for Class in labels:
    path="I:/New/"+Class
    path2="I:/SKULL/"+Class
    for name in os.listdir(path):
        I=cv2.imread(path+"/"+name)
        I=cv2.cvtColor(I,cv2.COLOR_BGR2RGB)
        I=np.array(cv2.resize(I,(128,128)))
        I=crop_imgs(I)
        I=resize_img(I,(128,128))
        I=I[:,:,1]
        cv2.imwrite(path2+"/"+"stipp_"+name,I)